In [37]:
import sys
import os
sys.path.append(os.path.abspath("."))  
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from src.data_pipeline import prepare_sequence
from src.data_processing import load_euroc_imu


In [ ]:
class IMUEncoder(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64, encoder_type="rnn"):
        super(IMUEncoder, self).__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.output_dim = hidden_dim * 2 
        self.norm = nn.LayerNorm(self.output_dim)
        self.dropout = nn.Dropout(0.3)


    def forward(self, x):
        _, (hn, _) = self.rnn(x)
        h = torch.cat((hn[-2], hn[-1]), dim=1)
        h = self.norm(h)
        h = self.dropout(h)
        return h


In [ ]:
sequence_length = 50
batch = prepare_sequence("data/data.csv", sequence_length)
model = IMUEncoder(input_dim=6, hidden_dim=64)
output = model(batch)